In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [162]:
PARTS_MASTER = pd.read_pickle('data/temp02/PARTS_MASTER.pkl')
ALTERNATES = pd.read_pickle('data/temp02/ALTERNATES.pkl')
PMA_ACCEPTANCE = pd.read_pickle('data/temp02/PMA_ACCEPTANCE.pkl')

#remove self equivalent PN
ALTERNATES = ALTERNATES[ALTERNATES['PNM'] != ALTERNATES['ALT_PNM']]

#remove duplicates
ALTERNATES.drop_duplicates(inplace = True)

#remove PN not in master
ALTERNATES = ALTERNATES[~ALTERNATES['PNM'].isnull()]
ALTERNATES = ALTERNATES[~ALTERNATES['ALT_PNM'].isnull()]

#all all PN as alternate for themselves
SIMPLE_PNM = PARTS_MASTER[['BEST_PNM','PN','CODE','TOTAL_SCORE']].copy()
SIMPLE_PNM.drop_duplicates(subset =['BEST_PNM'],inplace = True)
SIMPLE_PNM['ALT_PNM'] = SIMPLE_PNM['BEST_PNM']
SIMPLE_PNM['ALT_PN'] = SIMPLE_PNM['PN']
SIMPLE_PNM['ALT_CODE'] = SIMPLE_PNM['CODE']
SIMPLE_PNM.rename(columns={'BEST_PNM':'PNM','TOTAL_SCORE':'ALT_SCORE'},inplace = True)
ALTERNATES = ALTERNATES.append(SIMPLE_PNM,sort=False)
ALTERNATES.drop_duplicates(inplace = True)
ALTERNATES.sort_values(by='ALT_SCORE', ascending = False, inplace = True)

#bring top alt to parts_master (top alt is the generic class name)
TOP_ALT = ALTERNATES.drop_duplicates(subset=['PNM'],keep='first')
PARTS_MASTER = PARTS_MASTER.merge(TOP_ALT[['PNM','ALT_PNM','ALT_CODE']],left_on='BEST_PNM',right_on='PNM',how='left')
PARTS_MASTER.drop(columns=['PNM_y'],inplace = True)
PARTS_MASTER.rename(columns={'PNM_x':'PNM'},inplace = True)

#bring top OEM alt
TOP_OEM_ALT = TOP_ALT[TOP_ALT['ALT_CODE'] == 'OE']
TOP_ALT = ALTERNATES.drop_duplicates(subset=['PNM'],keep='first')
PARTS_MASTER = PARTS_MASTER.merge(TOP_OEM_ALT[['PNM','ALT_PNM']],left_on='BEST_PNM',right_on='PNM',how='left')
PARTS_MASTER.drop(columns=['PNM_y'],inplace = True)
PARTS_MASTER.rename(columns={'PNM_x':'PNM','ALT_PNM_x':'ALT_PNM','ALT_PNM_y':'ALT_PNM_OE'},inplace = True)

#bring top PMA alt
TOP_PMA_ALT = TOP_ALT[TOP_ALT['ALT_CODE'] == 'PM']
TOP_ALT = ALTERNATES.drop_duplicates(subset=['PNM'],keep='first')
PARTS_MASTER = PARTS_MASTER.merge(TOP_PMA_ALT[['PNM','ALT_PNM']],left_on='BEST_PNM',right_on='PNM',how='left')
PARTS_MASTER.drop(columns=['PNM_y'],inplace = True)
PARTS_MASTER.rename(columns={'PNM_x':'PNM','ALT_PNM_x':'ALT_PNM','ALT_PNM_y':'ALT_PNM_PM'},inplace = True)

#save dat file
PARTS_MASTER.to_pickle('data/temp03/PARTS_MASTER.pkl')